# Loan Approval Project

## Step 1: Historical Context Assessment

### Domain: Consumer lending / loan approvals

Loan approval systems are critical decision tools that affect access to credit, housing, and economic opportunity. Historical evidence shows that marginalized communities — especially by race, gender, and geography — have faced significant discrimination in lending practices.

### Documented patterns of harm

| Historical pattern                         | Source                                | Relevance to system                          |
|-------------------------------------------|---------------------------------------|----------------------------------------------|
| Redlining of minority neighborhoods       | U.S. HUD, CFPB Reports                | Loan access often tied to ZIP codes or geography |
| Gender bias in financial credit decisions | CFPB, Federal Reserve studies         | Women historically receive smaller loans, more denials |
| Algorithmic bias in credit risk models    | Research (e.g., Hurley & Adebayo, 2016) | ML systems may learn from biased historical labels |
| Lack of representation in training data   | Data & Society, AI Now Institute      | Underrepresentation can distort learned patterns |

### Implications

- Features such as **income**, **employment type**, or **property area** may act as proxies for protected attributes.
- Labels (i.e., past loan approvals) may reflect **systemic or subjective discrimination**.
- Model outcomes should be evaluated for fairness across **gender** and **region**, and **intersectional subgroups** (e.g., Female + Rural).

### Outputs to feed into:
- Fairness definition selection
- Bias source identification
- Metric prioritization


## Step 2: Fairness Definition Selection

### System impact

This loan approval system determines access to financial credit. The **harm** in this context is:
- Denial of opportunity to qualified applicants
- Reinforcement of historical disparities in credit access

Therefore, our fairness priorities focus on ensuring **equal access for equally qualified individuals** and **equity in approval rates**.

---

### Selected fairness definitions

| Definition             | Why it was selected                                  |
|------------------------|------------------------------------------------------|
| **Equal Opportunity**  | Ensures **true positive rate (TPR)** is similar across groups → qualified individuals are not unfairly denied. |
| **Demographic Parity** | Tracks whether **approval rates** reflect population fairness goals, even if not optimized for parity. |

These definitions were chosen to balance:
- **Legal defensibility** (Equal Opportunity relates to disparate impact standards)
- **Public fairness expectations** (representation fairness for approval outcomes)

---

### Application plan

- Use **True Positive Rate Difference** to measure Equal Opportunity.
- Use **Positive Prediction Rate Difference** for Demographic Parity.
- Where needed, extend to **intersectional subgroups** (e.g., Gender + Property Area).


## Step 3: Bias Source Identification

This section outlines where bias may enter the ML pipeline for loan approvals, from data collection through deployment.


### Pipeline bias map

| Stage               | Bias type             | Example for this project                                  | Risk? | Mitigation Idea |
|---------------------|------------------------|-----------------------------------------------------------|-------|------------------|
| Data Collection      | Representation Bias    | Under-sampling of rural or female applicants              | Medium | Stratified sampling or oversampling |
| Labeling             | Historical Bias        | Past approvals reflect biased human decisions             | High   | Reweight labels or debias targets |
| Feature Engineering  | Measurement Bias       | Income may encode systemic disparities                    | High   | Examine correlation with protected attributes |
| Model Optimization   | Optimization Bias      | Trained purely for accuracy, ignoring fairness constraints| Medium | Consider fair-aware objective or thresholds |
| Evaluation           | Aggregation Bias       | Overall accuracy may hide subgroup disparities            | High   | Always report per-group metrics |
| Deployment           | Feedback Loops         | Approved users influence future data                      | Low    | Monitor deployment skew |

---

### Next actions

- Analyze distribution of **Gender**, **Property_Area**, **LoanAmount**, etc.
- Check correlations between features and protected attributes
- Flag features that may act as **proxies for race, gender, or geography**
- Document all risks in final report

In [61]:
# Load and inspect the dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("fairness_audit_framework/example/LoanApprovalPrediction.csv")

# Display basic information
df.info()

# Preview the first few rows
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            598 non-null    object 
 1   Gender             598 non-null    object 
 2   Married            598 non-null    object 
 3   Dependents         586 non-null    float64
 4   Education          598 non-null    object 
 5   Self_Employed      598 non-null    object 
 6   ApplicantIncome    598 non-null    int64  
 7   CoapplicantIncome  598 non-null    float64
 8   LoanAmount         577 non-null    float64
 9   Loan_Amount_Term   584 non-null    float64
 10  Credit_History     549 non-null    float64
 11  Property_Area      598 non-null    object 
 12  Loan_Status        598 non-null    object 
dtypes: float64(5), int64(1), object(7)
memory usage: 60.9+ KB


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [62]:
print("Loan_Status unique values:", df["Loan_Status"].unique())
print("Gender unique values:", df["Gender"].unique())
print("Property_Area unique values:", df["Property_Area"].unique())

Loan_Status unique values: ['Y' 'N']
Gender unique values: ['Male' 'Female']
Property_Area unique values: ['Urban' 'Rural' 'Semiurban']


In [63]:
# Convert 'Loan_Status' to numeric: 1 for 'Y', 0 for 'N'
df["Loan_Status"] = df["Loan_Status"].map({"Y": 1, "N": 0})

# Re-encode categorical fields if needed (e.g., for grouping or modeling later)
# Not strictly required here, but useful for consistency
df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})
df["Property_Area"] = df["Property_Area"].map({"Rural": 0, "Urban": 1, "Semiurban": 2})

# Drop rows with any NA values in key columns
df_clean = df.dropna(subset=["Loan_Status", "Gender", "Property_Area"])

# Group-wise approval rate
approval_by_gender = df_clean.groupby("Gender")["Loan_Status"].mean()
approval_by_area = df_clean.groupby("Property_Area")["Loan_Status"].mean()

print("Approval Rate by Gender:")
print(approval_by_gender)

print("\nApproval Rate by Property Area:")
print(approval_by_area)

Approval Rate by Gender:
Gender
0    0.666667
1    0.691992
Name: Loan_Status, dtype: float64

Approval Rate by Property Area:
Property_Area
0    0.617143
1    0.656566
2    0.768889
Name: Loan_Status, dtype: float64


In [64]:
# Number of applications by Gender
gender_counts = df_clean["Gender"].value_counts()
print("Number of applications by Gender:")
print(gender_counts)

# Number of applications by Property Area
area_counts = df_clean["Property_Area"].value_counts()
print("\n Number of applications by Property Area:")
print(area_counts)

# Combined group Gender + Property_Area
df_clean["Group"] = df_clean["Gender"].astype(str) + "_" + df_clean["Property_Area"].astype(str)
group_counts = df_clean["Group"].value_counts()
print("\n Number of applications by Gender + Property Area:")
print(group_counts)

Number of applications by Gender:
1    487
0    111
Name: Gender, dtype: int64

 Number of applications by Property Area:
2    225
1    198
0    175
Name: Property_Area, dtype: int64

 Number of applications by Gender + Property Area:
1_2    171
1_1    165
1_0    151
0_2     54
0_1     33
0_0     24
Name: Group, dtype: int64


Note: 
Gender encoding: Male: 1, Female: 0
Area encoding: Rural: 0, Urban: 1, Semiurban: 2

In [65]:
# Check feature correlation with Loan_Status
corr = df_clean.select_dtypes(include="number").corr()
print("Feature correlation with Loan_Status:")
print(corr["Loan_Status"].sort_values(ascending=False))

Feature correlation with Loan_Status:
Loan_Status          1.000000
Credit_History       0.557308
Property_Area        0.135827
Gender               0.021239
Dependents           0.003048
Loan_Amount_Term    -0.017554
ApplicantIncome     -0.025248
LoanAmount          -0.055643
CoapplicantIncome   -0.058194
Name: Loan_Status, dtype: float64


##### Insights

The data suggests **representation bias**. Male applicants make up ~81% of the dataset. Female applicants, especially in rural areas, are severely underrepresented (just 24 samples). 

Property area is fairly balanced, but Semiurban is slightly overrepresented.

In terms of feature correclation, credit history dominates prediction. This may encode **historical bias** if not audited carefully.

##### Key implications

| Pipeline Stage   | Potential Bias Source                       | Mitigation Recommendation                         |
|------------------|---------------------------------------------|---------------------------------------------------|
| Data Collection  | Underrepresentation of women & rural groups | Stratify datasets, resample or augment data       |
| Feature Design   | Credit history dominance                    | Audit for proxy discrimination                    |
| Labeling         | Labels reflect historical approvals         | Assess for historical discrimination in labels    |
| Evaluation       | Group metrics affected by small-N           | Use confidence intervals and intersectional views |


Let's proceed with fairness metric evaluation next and incorporate these findings to inform our metric selection and interpretation.

## Train model

#### Preprocess the dataset
- Remove irrelevant identifiers like Loan_ID.
- Fill in missing values using median (for numerical) and mode (for categorical).
- Encode remaining categorical features
- Feature scaling using StandardScaler to standardize features before feeding into logistic regression. This prevents features with large ranges (e.g., income) from dominating the model
- Split the dataset by Loan_Status to preserve class balance in both train and test sets  

In [66]:
# ## Step 2: Data Cleaning and Preprocessing

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Drop Loan_ID (not useful for modeling)
df.drop('Loan_ID', axis=1, inplace=True)

# Fill missing values
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['LoanAmount'].fillna(df['LoanAmount'].median(), inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)

# Encode categorical variables
categorical_cols = ['Married', 'Education', 'Self_Employed']
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Define features and target
X = df.drop('Loan_Status', axis=1)
y = df['Loan_Status']

# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split stratified by the target label
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # Maintain class balance
)

# Confirm shape and class distribution
print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")
print("Class distribution in y_train:", np.bincount(y_train))
print("Class distribution in y_test:", np.bincount(y_test))

Training samples: 478, Test samples: 120
Class distribution in y_train: [149 329]
Class distribution in y_test: [38 82]


##### Train a baseline model using logistic regression and assess the model’s performance using accuracy, confusion matrix, and classification metrics

In [67]:
# ## Step 3: Train and Evaluate Logistic Regression Model

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Train logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Output results
print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", report)


Accuracy: 0.8083333333333333

Confusion Matrix:
 [[17 21]
 [ 2 80]]

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.45      0.60        38
           1       0.79      0.98      0.87        82

    accuracy                           0.81       120
   macro avg       0.84      0.71      0.74       120
weighted avg       0.82      0.81      0.79       120



## Step 4: Fairness Metrics Evaluation

In [68]:
# Build a DataFrame to analyze results

X_test_df = pd.DataFrame(X_test, columns=X.columns).reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_test_df["y_true"] = y_test

In [72]:
from sklearn.metrics import confusion_matrix

# Reconstruct original group info
X_test_df = pd.DataFrame(X_test, columns=X.columns)
X_test_df["Gender"] = df.loc[X_test_df.index, "Gender"].values
X_test_df["Property_Area"] = df.loc[X_test_df.index, "Property_Area"].values
X_test_df["y_true"] = y_test
X_test_df["y_pred"] = y_pred

# Define helper functions
def compute_tpr(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        return tp / (tp + fn) if (tp + fn) > 0 else 0
    return 0

def compute_positive_rate(y_pred):
    return np.mean(y_pred)

# Group-wise fairness analysis
print("\nFairness by Gender:")
for gender in sorted(X_test_df["Gender"].unique()):
    group = X_test_df[X_test_df["Gender"] == gender]
    tpr = compute_tpr(group["y_true"], group["y_pred"])
    ppr = compute_positive_rate(group["y_pred"])
    print(f"Gender {gender} -> TPR: {round(tpr, 3)} | Positive Rate: {round(ppr, 3)} | Samples: {len(group)}")

print("\nFairness by Property_Area:")
for area in sorted(X_test_df["Property_Area"].unique()):
    group = X_test_df[X_test_df["Property_Area"] == area]
    tpr = compute_tpr(group["y_true"], group["y_pred"])
    ppr = compute_positive_rate(group["y_pred"])
    print(f"Area {area} -> TPR: {round(tpr, 3)} | Positive Rate: {round(ppr, 3)} | Samples: {len(group)}")

# Intersectional fairness
X_test_df["Group"] = X_test_df["Gender"].astype(str) + "_" + X_test_df["Property_Area"].astype(str)
print("\nIntersectional Fairness:")
for group_id in sorted(X_test_df["Group"].unique()):
    group = X_test_df[X_test_df["Group"] == group_id]
    tpr = compute_tpr(group["y_true"], group["y_pred"])
    ppr = compute_positive_rate(group["y_pred"])
    print(f"Group {group_id} -> TPR: {round(tpr, 3)} | Positive Rate: {round(ppr, 3)} | Samples: {len(group)}")



Fairness by Gender:
Gender 0 -> TPR: 1.0 | Positive Rate: 0.9 | Samples: 20
Gender 1 -> TPR: 0.97 | Positive Rate: 0.83 | Samples: 100

Fairness by Property_Area:
Area 0 -> TPR: 0.889 | Positive Rate: 0.846 | Samples: 13
Area 1 -> TPR: 0.975 | Positive Rate: 0.828 | Samples: 58
Area 2 -> TPR: 1.0 | Positive Rate: 0.857 | Samples: 49

Intersectional Fairness:
Group 0_1 -> TPR: 1.0 | Positive Rate: 0.833 | Samples: 6
Group 0_2 -> TPR: 1.0 | Positive Rate: 0.929 | Samples: 14
Group 1_0 -> TPR: 0.889 | Positive Rate: 0.846 | Samples: 13
Group 1_1 -> TPR: 0.971 | Positive Rate: 0.827 | Samples: 52
Group 1_2 -> TPR: 1.0 | Positive Rate: 0.829 | Samples: 35


### 5. Audit Gap and recommendations

After evaluating fairness metrics across gender, property area, and intersectional subgroups, several **limitations** and **gaps** remain that could obscure or distort audit conclusions.

---

#### Gap 1: Incomplete subgroup evaluation

Some subgroups (e.g., female + rural) had **low sample counts** in the test set.

- **Example**: "Group 0_1" has only 6 samples — resulting in potentially **unstable TPR or Positive Rate** estimates.
- **Implication**: Fairness conclusions based on small samples may not generalize.

**Next Steps**:
- Ensure stratified test sampling retains subgroup diversity.
- Consider bootstrapped confidence intervals or Bayesian estimation for low-n cases.

---

#### Gap 2: No confidence Intervals or statistical significance

All fairness metrics (TPR, Positive Rate) were point estimates.

- **Implication**: Without confidence intervals, we don’t know whether metric differences are **statistically meaningful** or due to noise.

**Next Steps**:
- Apply bootstrapping or use tools like `fairlearn` or `AIF360` to compute confidence intervals.
- Use multiple testing corrections (e.g., Benjamini-Hochberg) when evaluating many metrics.

---

#### Gap 3: Feature-level fairness not assessed

We assessed output disparities but not **feature-level sources** of potential bias.

- **Examples**: Credit history, loan amount, income may have disparate impact.
- **Implication**: Bias may be hidden in model inputs even if output metrics look balanced.

**Next Steps**:
- Perform fairness analysis for individual features.
- Check for proxies of protected attributes or skewed distributions.

---

#### Gap 4: No label quality audit

The label `Loan_Status` was assumed accurate and unbiased.

- **Implication**: If historical decisions used to create labels were biased, the model may inherit these biases.

**Next Steps**:
- Trace label generation process for known human or systemic bias.
- Consider re-labeling or debiasing approaches if justified.

---

#### Gap 5: No uncertainty or drift evaluation

The audit was performed on a single snapshot of data.

- **Implication**: Fairness performance might degrade over time (temporal drift).

**Next Steps**:
- Schedule recurring audits post-deployment.
- Monitor group-wise metrics over time.

---

### Summary

| Area                   | Gap Identified                            | Recommendation                                  |
|------------------------|-------------------------------------------|-------------------------------------------------|
| Subgroup coverage      | Low samples for some groups               | Stratified sampling + CIs                       |
| Metric uncertainty     | No statistical bounds                     | Bootstrap or `fairlearn` tools                  |
| Feature inputs         | Not audited for fairness impact           | Add feature-level disparity checks              |
| Label bias             | Not verified                              | Review or re-label based on human decision bias |
| Generalizability       | Single data slice                         | Add recurring evaluations and drift checks      |

